In [14]:
from neo4j import GraphDatabase
import pandas as pd

# Connect to Neo4j
driver = GraphDatabase.driver('neo4j://212.235.239.171:22333', auth=('neo4j', 'ngslbd2020'))

def read_query(query, params={}):
    with driver.session(database="semmed432202") as session:
        result = session.run(query, params)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())

In [23]:
data = read_query("""
MATCH (a)-[r:CAUSES|AFFECTS|STIMULATES|INHIBITS|AUGMENTS|PREDISPOSES|ASSOCIATED_WITH]->(b)
WHERE a.is_hpo = True AND b.is_hpo = True
WITH split(a.cui, "|") AS as, split(b.cui, "|") AS bs, r
UNWIND as AS ai
UNWIND bs AS bi
RETURN DISTINCT ai as source, bi as target, type(r) as type
""")

In [24]:
len(data)

974361

In [25]:
data.head()

,source,target,type
0,1,2048,STIMULATES
1,1,5594,STIMULATES
2,1,C0001807,ASSOCIATED_WITH
3,1,C0007222,ASSOCIATED_WITH
4,1,C0009324,ASSOCIATED_WITH


In [26]:
!pip show pykeen

Name: pykeen
Version: 1.8.2
Summary: A package for training and evaluating multimodal knowledge graph embeddings
Home-page: https://github.com/pykeen/pykeen
Author: "Mehdi Ali, Max Berrendorf, Charles Tapley Hoyt, Laurent Vermue"
Author-email: pykeen2019@gmail.com
License: MIT
Location: c:\users\tomas\anaconda3\lib\site-packages
Requires: pyyaml, pystow, docdata, rexmex, dataclasses-json, numpy, pandas, optuna, torch-ppr, more-itertools, scipy, requests, tqdm, sklearn, click-default-group, click, tabulate, more-click, torch-max-mem, class-resolver
Required-by: 


In [27]:
from pykeen.triples import TriplesFactory

tf = TriplesFactory.from_labeled_triples(
  data[["source", "type", "target"]].values
)

In [28]:
training, testing, validation = tf.split([.8, .1, .1])


INFO:pykeen.triples.splitting:done splitting triples to groups of sizes [741806, 97436, 97437]


In [29]:
from pykeen.pipeline import pipeline

result = pipeline(
    training=training,
    testing=testing,
    validation=validation,
    model='PairRE',
    stopper='early',
    epochs=500,
    dimensions=512,
    random_seed=420
)

INFO:pykeen.pipeline.api:Using device: None
C:\Users\tomas\Anaconda3\lib\site-packages\pykeen\nn\representation.py:373: UserWarning: Directly use Embedding.max_id instead of num_embeddings.
  warnings.warn(f"Directly use {self.__class__.__name__}.max_id instead of num_embeddings.")


Training epochs on cuda:0:   0%|          | 0/500 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/3045 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/3045 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/3045 [00:00<?, ?batch/s]

...

Training batches on cuda:0:   0%|          | 0/3045 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/3045 [00:00<?, ?batch/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 139.78s seconds
INFO:pykeen.stoppers.early_stopping:Stopping early at epoch 110. The best result 0.10476513028931515 occurred at epoch 90.


Evaluating on cuda:0:   0%|          | 0.00/97.4k [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 138.00s seconds


In [30]:
result.get_metric('inverse_geometric_mean_rank')

0.005394322331994772

In [31]:
result.save_to_directory('/pairre-small')

INFO:pykeen.triples.triples_factory:Stored TriplesFactory(num_entities=37859, num_relations=7, create_inverse_triples=False, num_triples=779488) to file:///C:/pairre-small/training_triples
INFO:pykeen.pipeline.api:Saved to directory: file:///C:/pairre-small


In [32]:
data.to_csv('pairre_data.csv', index=False)